# TFX Guided Project on Vertex

**Learning Objectives:**

* Learn how to generate a standard TFX template pipeline using `tfx template`
* Learn how to modify and run a templated TFX pipeline 

**Note:** This guided project is adapted from [Create a TFX pipeline using templates](https://www.tensorflow.org/tfx/tutorials/tfx/template)).

In [14]:
import os

from google.cloud import aiplatform

## Step 1. Environment setup

### `skaffold` tool setup

In [ ]:
# PATH = %env PATH
#%env PATH={PATH}:/home/jupyter/.local/bin

In [ ]:
#%%bash

# LOCAL_BIN="/home/jupyter/.local/bin"
# SKAFFOLD_URI="https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64"

# test -d $LOCAL_BIN || mkdir -p $LOCAL_BIN

# which skaffold || (
#    curl -Lo skaffold $SKAFFOLD_URI &&
#    chmod +x skaffold               &&
#    mv skaffold $LOCAL_BIN
# )

Modify the `PATH` environment variable so that `skaffold` is available:

At this point, you shoud see the `skaffold` tool with the command `which`:

In [ ]:
#!which skaffold

### Environment variable setup

In AI Platform Pipelines, TFX is running in a hosted Kubernetes environment using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/).

Let's set some environment variables to use Kubeflow Pipelines.

First, get your GCP project ID.

In [2]:
shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT = shell_output[0]
REGION = "us-central1"

%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
%env REGION={REGION}

env: GOOGLE_CLOUD_PROJECT=dherin-dev
env: REGION=us-central1


We also need to access your KFP cluster. You can access it in your Google Cloud Console under "AI Platform > Pipeline" menu.

The "endpoint" of the KFP cluster can be found from the URL of the Pipelines dashboard, 
or you can get it from the URL of the Getting Started page where you launched this notebook.

Let's create an ENDPOINT environment variable and set it to the KFP cluster endpoint.

ENDPOINT should contain only the hostname part of the URL. 
For example, if the URL of the KFP dashboard is

<a href="https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start">https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start</a>, 

ENDPOINT value becomes 1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com.

Set the image name as tfx-pipeline under the current GCP project:

In [ ]:
gcr.io / dherin - dev / tfx - guided - project - on - vertex

## Step 2. Copy the predefined template to your project directory.

In [ ]:
PIPELINE_NAME = "tfx-guided-project-on-vertex"

In this step, we will create a working pipeline project directory and 
files by copying additional files from a predefined template.

You may give your pipeline a different name by changing the PIPELINE_NAME below. 

This will also become the name of the project directory where your files will be put.

In [4]:
PROJECT_DIR = os.path.join(os.path.expanduser("."), PIPELINE_NAME)
PROJECT_DIR

'./tfx-guided-project-on-vertex'

TFX includes the taxi template with the TFX python package. 

If you are planning to solve a point-wise prediction problem,
including classification and regresssion, this template could be used as a starting point.

The `tfx template copy` CLI command copies predefined template files into your project directory.

In [5]:
!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=taxi

CLI
Copying taxi pipeline template
kubeflow_v2_runner.py -> ./tfx-guided-project-on-vertex/kubeflow_v2_runner.py
__init__.py -> ./tfx-guided-project-on-vertex/__init__.py
__init__.py -> ./tfx-guided-project-on-vertex/models/__init__.py
__init__.py -> ./tfx-guided-project-on-vertex/models/estimator_model/__init__.py
constants.py -> ./tfx-guided-project-on-vertex/models/estimator_model/constants.py
model.py -> ./tfx-guided-project-on-vertex/models/estimator_model/model.py
model_test.py -> ./tfx-guided-project-on-vertex/models/estimator_model/model_test.py
preprocessing_test.py -> ./tfx-guided-project-on-vertex/models/preprocessing_test.py
features.py -> ./tfx-guided-project-on-vertex/models/features.py
preprocessing.py -> ./tfx-guided-project-on-vertex/models/preprocessing.py
__init__.py -> ./tfx-guided-project-on-vertex/models/keras_model/__init__.py
constants.py -> ./tfx-guided-project-on-vertex/models/keras_model/constants.py
model.py -> ./tfx-guided-project-on-vertex/models/keras_mod

In [18]:
# Docker image name for the pipeline image.
# CUSTOM_TFX_IMAGE = "gcr.io/" + GOOGLE_CLOUD_PROJECT + "/tfx-pipeline"
CUSTOM_TFX_IMAGE = f"gcr.io/{GOOGLE_CLOUD_PROJECT}/{PIPELINE_NAME}"
CUSTOM_TFX_IMAGE

'gcr.io/dherin-dev/tfx-guided-project-on-vertex'

In [6]:
%cd {PROJECT_DIR}

/home/jupyter/asl-ml-immersion/notebooks/tfx_pipelines/guided_projects/tfx-guided-project-on-vertex


In [20]:
%%writefile Dockerfile
FROM gcr.io/tfx-oss-public/tfx:1.4.0

RUN pip install -U pip
RUN pip install google-cloud-aiplatform==1.7.1 kfp==1.8.1


COPY . ./

Writing Dockerfile


In [21]:
!gcloud builds submit --timeout 15m --tag $CUSTOM_TFX_IMAGE .

Creating temporary tarball archive of 35 file(s) totalling 1.9 MiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/jupyter/.config/gcloud/logs/2022.01.24/13.54.23.488051.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://dherin-dev_cloudbuild/source/1643032463.596892-1ac908d7d7854f549a994f46e3fbc436.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/dherin-dev/locations/global/builds/0fe1c53a-02b4-46a4-9948-fcae6e63a10a].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/0fe1c53a-02b4-46a4-9948-fcae6e63a10a?project=115851500182].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "0fe1c53a-02b4-46a4-9948-fcae6e63a10a"

FETCHSOURCE
Fetching storage object: gs://dherin-dev_cloudbuild/source/1643032463.596892-1ac908d7d7854f549a994f46e3fbc436.tgz#16

### Step 3. Browse your copied source files

The TFX template provides basic scaffold files to build a pipeline, including Python source code,
sample data, and Jupyter Notebooks to analyse the output of the pipeline. 

The `taxi` template uses the Chicago Taxi dataset.

Here is brief introduction to each of the Python files:

`pipeline` - This directory contains the definition of the pipeline
* `configs.py` — defines common constants for pipeline runners
* `pipeline.py` — defines TFX components and a pipeline

`models` - This directory contains ML model definitions.
* `features.py`, `features_test.py` — defines features for the model
* `preprocessing.py`, `preprocessing_test.py` — defines preprocessing jobs using tf::Transform

`models/estimator` - This directory contains an Estimator based model.
* `constants.py` — defines constants of the model
* `model.py`, `model_test.py` — defines DNN model using TF estimator

`models/keras` - This directory contains a Keras based model.
* `constants.py` — defines constants of the model
* `model.py`, `model_test.py` — defines DNN model using Keras

`local_runner.py`, `kubeflow_runner.py`, `kubeflow_v2_runner.py` — define runners for each orchestration engine


**Running the tests:**
You might notice that there are some files with `_test.py` in their name. 
These are unit tests of the pipeline and it is recommended to add more unit 
tests as you implement your own pipelines. 
You can run unit tests by supplying the module name of test files with `-m` flag. 
You can usually get a module name by deleting `.py` extension and replacing `/` with `..`

For example:

In [7]:
!python -m models.features_test
!python -m models.keras.model_test

Running tests under Python 3.7.12: /opt/conda/bin/python
[ RUN      ] FeaturesTest.testNumberOfBucketFeatureBucketCount
INFO:tensorflow:time(__main__.FeaturesTest.testNumberOfBucketFeatureBucketCount): 0.0s
I0124 13:38:59.879367 139948156094272 test_util.py:2189] time(__main__.FeaturesTest.testNumberOfBucketFeatureBucketCount): 0.0s
[       OK ] FeaturesTest.testNumberOfBucketFeatureBucketCount
[ RUN      ] FeaturesTest.testTransformedNames
INFO:tensorflow:time(__main__.FeaturesTest.testTransformedNames): 0.0s
I0124 13:38:59.879930 139948156094272 test_util.py:2189] time(__main__.FeaturesTest.testTransformedNames): 0.0s
[       OK ] FeaturesTest.testTransformedNames
[ RUN      ] FeaturesTest.test_session
[  SKIPPED ] FeaturesTest.test_session
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK (skipped=1)
/opt/conda/bin/python: Error while finding module specification for 'models.keras.model_test' (ModuleNotFoundError: No module named 'model

Let's quickly go over the structure of a test file to test Tensorflow code:

In [8]:
!tail -26 models/features_test.py

# See the License for the specific language governing permissions and
# limitations under the License.


import tensorflow as tf

from models import features


class FeaturesTest(tf.test.TestCase):

  def testNumberOfBucketFeatureBucketCount(self):
    self.assertEqual(
        len(features.BUCKET_FEATURE_KEYS),
        len(features.BUCKET_FEATURE_BUCKET_COUNT))
    self.assertEqual(
        len(features.CATEGORICAL_FEATURE_KEYS),
        len(features.CATEGORICAL_FEATURE_MAX_VALUES))

  def testTransformedNames(self):
    names = ["f1", "cf"]
    self.assertEqual(["f1_xf", "cf_xf"], features.transformed_names(names))


if __name__ == "__main__":
  tf.test.main()


First of all, notice that you start by importing the code you want to test by importing the corresponding module. Here we want to test the code in `features.py` so we import the module `features`:
```python
from models import features
```
To implement test cases start by defining your own test class inheriting from `tf.test.TestCase`:
```python
class FeaturesTest(tf.test.TestCase):
```
Wen you execute the test file with
```bash
python -m models.features_test
```
the main method
```python
 tf.test.main()
```
will parse your test class (here: `FeaturesTest`) and execute every method whose name starts by `test`. Here we have two such methods for instance:
```python
def testNumberOfBucketFeatureBucketCount(self):
def testTransformedNames(self):
```
So when you want to add a test case, just add a method to that test class whose name starts by `test`. Now inside the body of these test methods is where the actual testing takes place. In this case for instance, `testTransformedNames` test the function `features.transformed_name` and makes sure it outputs what is expected.
Since your test class inherits from `tf.test.TestCase` it has a number of helper methods you can use to help you create tests, as for instance
```python
self.assertEqual(expected_outputs, obtained_outputs)
```
that will fail the test case if `obtained_outputs` do the match the `expected_outputs`. 


Typical examples of test case you may want to implement for machine learning code would comprise test insurring that your model builds correctly, your preprocessing function preprocesses raw data as expected, or that your model can train successfully on a few mock examples. When writing tests make sure that their execution is fast (we just want to check that the code works not actually train a performant model when testing). For that you may have to create synthetic data in your test files. For more information, read the [tf.test.TestCase documentation](https://www.tensorflow.org/api_docs/python/tf/test/TestCase) and the [Tensorflow testing best practices](https://www.tensorflow.org/community/contribute/tests).


## Step 4. Run your first TFX pipeline

Components in the TFX pipeline will generate outputs for each run as
[ML Metadata Artifacts](https://www.tensorflow.org/tfx/guide/mlmd), and they need to be stored somewhere.
You can use any storage which the KFP cluster can access, and for this example we
will use Google Cloud Storage (GCS).

Let us create this bucket. Its name will be `<YOUR_PROJECT>-kubeflowpipelines-default`.

In [25]:
GCS_BUCKET_NAME = GOOGLE_CLOUD_PROJECT + "-kubeflowpipelines-default"
GCS_BUCKET_NAME

'dherin-dev-kubeflowpipelines-default'

In [26]:
!gsutil ls | grep ^gs://{GCS_BUCKET_NAME}/$ || gsutil mb -l {REGION} gs://{GCS_BUCKET_NAME}

gs://dherin-dev-kubeflowpipelines-default/


Let's upload our sample data to GCS bucket so that we can use it in our pipeline later.

In [28]:
!gsutil cp data/data.csv gs://{GCS_BUCKET_NAME}/tfx-template/data/taxi/data.csv

Copying file://data/data.csv [Content-Type=text/csv]...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      


Let's create a TFX pipeline using the `tfx pipeline create` command.

**Note:** When creating a pipeline for KFP, we need a container image which will 
be used to run our pipeline. And skaffold will build the image for us. Because `skaffold`
pulls base images from the docker hub, it will take 5~10 minutes when we build
the image for the first time, but it will take much less time from the second build.

In [12]:
#!tfx pipeline create  \
# --pipeline-path=kubeflow_dag_runner.py \
# --endpoint={ENDPOINT} \
# --build-target-image={CUSTOM_TFX_IMAGE}

In [30]:
!tfx pipeline compile --engine vertex --pipeline_path kubeflow_v2_runner.py

CLI
Compiling pipeline
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.ty

While creating a pipeline, `Dockerfile` and `build.yaml` will be generated to build a Docker image.

Don't forget to add these files to the source control system (for example, git) along with other source files.

A pipeline definition file for [argo](https://argoproj.github.io/argo/) will be generated, too. 
The name of this file is `${PIPELINE_NAME}.tar.gz.` 
For example, it will be `guided_project_1.tar.gz` if the name of your pipeline is `guided_project_1`. 
It is recommended NOT to include this pipeline definition file into source control, because it will be generated from other Python files and will be updated whenever you update the pipeline. For your convenience, this file is already listed in `.gitignore` which is generated automatically.

Now start an execution run with the newly created pipeline using the `tfx run create` command.

**Note:** You may see the following error `Error importing tfx_bsl_extension.coders.` Please ignore it.

**Debugging tip:** If your pipeline run fails, you can see detailed logs for each TFX component in the Experiments tab in the KFP Dashboard. One of the major sources of failure is **permission related problems**. 
Please make sure your KFP cluster has permissions to access Google Cloud APIs.
This can be configured [when you create a KFP cluster in GCP](https://cloud.google.com/ai-platform/pipelines/docs/setting-up),
or see [Troubleshooting document in GCP](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting).

In [ ]:
#!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

In [31]:
aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path="pipeline.json",
    enable_caching=True,
)

pipeline.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124202438
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124202438')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tfx-guided-project-on-vertex-20220124202438?project=115851500182
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124202438 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/1

Or, you can also run the pipeline in the KFP Dashboard. The new execution run will be listed 
under Experiments in the KFP Dashboard. 
Clicking into the experiment will allow you to monitor progress and visualize 
the artifacts created during the execution run.

However, we recommend visiting the KFP Dashboard. You can access the KFP Dashboard from 
the Cloud AI Platform Pipelines menu in Google Cloud Console. Once you visit the dashboard, 
you will be able to find the pipeline, and access a wealth of information about the pipeline. 
For example, you can find your runs under the Experiments menu, and when you open your
execution run under Experiments you can find all your artifacts from the pipeline under Artifacts menu.

## Step 5. Add components for data validation.

In this step, you will add components for data validation including `StatisticsGen`, `SchemaGen`, and `ExampleValidator`.
If you are interested in data validation, please see 
[Get started with Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

**Double-click to change directory to pipeline and double-click again to open** `pipeline.py`. 
Find and uncomment the 3 lines which add `StatisticsGen`, `SchemaGen`, and `ExampleValidator` to the pipeline.
(Tip: search for comments containing TODO(step 5):). Make sure to save `pipeline.py` after you edit it.

You now need to update the existing pipeline with modified pipeline definition. Use the `tfx pipeline update` command to update your pipeline, followed by the `tfx run create` command to create a new execution run of your updated pipeline.

In [32]:
!tfx pipeline compile --engine vertex --pipeline_path kubeflow_v2_runner.py

CLI
Compiling pipeline
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.ty

In [33]:
aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path="pipeline.json",
    enable_caching=True,
)

pipeline.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124205225
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124205225')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tfx-guided-project-on-vertex-20220124205225?project=115851500182
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124205225 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/1

### Check pipeline outputs

Visit the KFP dashboard to find pipeline outputs in the page for your pipeline run. Click the Experiments tab on the left, and All runs in the Experiments page. You should be able to find the latest run under the name of your pipeline.

See link below to access the dashboard:

In [ ]:
print("https://" + ENDPOINT)

## Step 6. Add components for training

In this step, you will add components for training and model validation including `Transform`, `Trainer`, `Resolver`, `Evaluator`, and `Pusher`.

**Double-click to open** `pipeline.py`. Find and uncomment the 5 lines which add `Transform`, `Trainer`, `ResolverNode`, `Evaluator` and `Pusher` to the pipeline. (Tip: search for TODO(step 6):)

As you did before, you now need to update the existing pipeline with the modified pipeline definition. The instructions are the same as Step 5. Update the pipeline using `tfx pipeline update`, and create an execution run using `tfx run create`.

Verify that the pipeline DAG has changed accordingly in the Kubeflow UI:

In [34]:
!tfx pipeline compile --engine vertex --pipeline_path kubeflow_v2_runner.py

CLI
Compiling pipeline
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.ty

In [ ]:
aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path="pipeline.json",
    enable_caching=True,
)

pipeline.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124210026
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124210026')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tfx-guided-project-on-vertex-20220124210026?project=115851500182
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220124210026 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/1

When this execution run finishes successfully, you have now created and run your first TFX pipeline in AI Platform Pipelines!

## Step 7. Try BigQueryExampleGen

[BigQuery](https://cloud.google.com/bigquery) is a serverless, highly scalable, and cost-effective cloud data warehouse.
`BigQuery` can be used as a source for training examples in TFX. In this step, we will add `BigQueryExampleGen` to the pipeline.

**Double-click to open** `pipeline.py`. Comment out `CsvExampleGen` and uncomment the line which creates an instance of `BigQueryExampleGen`. You also need to uncomment the query argument of the `create_pipeline` function.

We need to specify which GCP project to use for `BigQuery`, and this is done by setting `--project` in `beam_pipeline_args` when creating a pipeline.

**Double-click to open** `configs.py`. Uncomment the definition of `GOOGLE_CLOUD_REGION`, `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` and `BIG_QUERY_QUERY`. You should replace the region value in this file with the correct values for your GCP project.

**Note:** You MUST set your GCP region in the `configs.py` file before proceeding

**Change directory one level up.** Click the name of the directory above the file list. The name of the directory is the name of the pipeline which is `tfx-guided-project-on-vertex` if you didn't change.

**Double-click to open** `kubeflow_v2_runner.py`. Uncomment two arguments, `query` and `beam_pipeline_args`, for the `create_pipeline` function.

Now the pipeline is ready to use `BigQuery` as an example source. Update the pipeline as before and create a new execution run as we did in step 5 and 6.

In [37]:
!tfx pipeline compile --engine vertex --pipeline_path kubeflow_v2_runner.py

CLI
Compiling pipeline
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.ty

In [ ]:
aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path="pipeline.json",
    enable_caching=True,
)

pipeline.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220125110827
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220125110827')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tfx-guided-project-on-vertex-20220125110827?project=115851500182
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220125110827 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/1

## Step 8. Try Dataflow with KFP

Several [TFX Components uses Apache Beam](https://www.tensorflow.org/tfx/guide/beam) to implement data-parallel pipelines, and it means that you can distribute data processing workloads using [Google Cloud Dataflow](https://cloud.google.com/dataflow/). In this step, we will set the Kubeflow orchestrator to use dataflow as the data processing back-end for Apache Beam.

**Double-click pipeline to change directory, and double-click to open** `configs.py`. Uncomment the definition of `GOOGLE_CLOUD_REGION`, and `DATAFLOW_BEAM_PIPELINE_ARGS`.

**Double-click to open** `pipeline.py`. Change the value of enable_cache to False.

**Change directory one level up.** Click the name of the directory above the file list. The name of the directory is the name of the pipeline which is `tfx-guided-project-on-vertex` if you didn't change.

**Double-click to open** `kubeflow_v2_runner.py`. Uncomment `beam_pipeline_args`. (Also make sure to comment out current `beam_pipeline_arg`s that you added in Step 7.)

Note that we deliberately disabled caching. Because we have already run the pipeline successfully, we will get cached execution result for all components if cache is enabled.

Now the pipeline is ready to use Dataflow. Update the pipeline and create an execution run as we did in step 5 and 6.



In [41]:
!tfx pipeline compile --engine vertex --pipeline_path kubeflow_v2_runner.py

CLI
Compiling pipeline
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.ty

In [42]:
aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path="pipeline.json",
    enable_caching=False,
)

pipeline.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220125115325
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220125115325')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tfx-guided-project-on-vertex-20220125115325?project=115851500182
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/115851500182/locations/us-central1/pipelineJobs/tfx-guided-project-on-vertex-20220125115325 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/1

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [Transform].; Job (project_id = dherin-dev, job_id = 5049067345910169600) is failed due to the above error.; Failed to handle the job: {project_number = 115851500182, job_id = 5049067345910169600}"


You can find your Dataflow jobs in [Dataflow in Cloud Console](http://console.cloud.google.com/dataflow).

Please reset `enable_cache` to `True` to benefit from caching execution results.



**Double-click to open** `pipeline.py`. Reset the value of enable_cache to True.



## Step 9. Try Cloud AI Platform Training and Prediction with KFP

TFX interoperates with several managed GCP services, such as [Cloud AI Platform for Training and Prediction](https://cloud.google.com/ai-platform/). You can set your `Trainer` component to use Cloud AI Platform Training, a managed service for training ML models. Moreover, when your model is built and ready to be served, you can push your model to Cloud AI Platform Prediction for serving. In this step, we will set our `Trainer` and `Pusher` component to use Cloud AI Platform services.

Before editing files, you might first have to enable AI Platform Training & Prediction API.


**Double-click pipeline to change directory, and double-click to open** `configs.py`. Uncomment the definition of `GOOGLE_CLOUD_REGION`, `GCP_AI_PLATFORM_TRAINING_ARGS` and `GCP_AI_PLATFORM_SERVING_ARGS`. We will use our custom built container image to train a model in Cloud AI Platform Training, so we should set `masterConfig.imageUri` in `GCP_AI_PLATFORM_TRAINING_ARGS` to the same value as `CUSTOM_TFX_IMAGE` above.

**Change directory one level up, and double-click to open** `kubeflow_dag_runner.py`. Uncomment `ai_platform_training_args` and `ai_platform_serving_args`.

Update the pipeline and create an execution run as we did in step 5 and 6.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}

In [ ]:
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

You can find your training jobs in [Cloud AI Platform Jobs](https://console.cloud.google.com/ai-platform/jobs). If your pipeline completed successfully, you can find your model in [Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models).

## License

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.